In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

In [8]:
pd.options.display.max_columns = 999

In [9]:
df_final = pd.read_csv('arquivos\dadosalunos-21-08-2023.csv')

In [74]:
df_cursos = pd.read_csv('arquivos\cursos.csv')

In [75]:
df_estados = pd.read_csv('arquivos\estados.csv', usecols=['codigo_uf', 'uf'])
df_municipios = pd.read_csv('arquivos\municipios.csv', usecols=['nome', 'codigo_uf'])

df_municipios['nome'] = df_municipios['nome'].str.upper().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

df_estados_municipios = df_municipios.merge(df_estados, on='codigo_uf', how='left')

In [12]:
colunas_loc_nasc_aluno = ['CD_ORIGEM_ALUNO', 'LT_NACIONALIDADE', 'NM_PAIS_NASC', 'SG_UF_NASC_ALUNO', 'NM_CIDADE_NASC', 'SG_UF_FERIAS', 'NM_CIDADE_FERIAS']

In [13]:
colunas_a_remover = []

In [14]:
df_final[(df_final['DT_NASC_ALUNO'] == '2003-06-16') & (df_final['CD_CURSO'] == 26)]

,ANO_OCORRENCIA,CD_ALUNO,CD_CURSO,CD_ORIGEM_ALUNO,CD_TURNO,DE_NATURALIDADE,DH_MATRICULA,DH_SITUACAO,DT_ANO_INGRES,DT_NASC_ALUNO,LT_ESTADO_CIVIL,LT_MODALIDADE,LT_NACIONALIDADE,LT_RACA,NM_CIDADE_AULA,NM_CIDADE_EST2GRAU,NM_CIDADE_FERIAS,NM_CIDADE_NASC,NM_PAIS_NASC,NU_SEMESTRE2_VEST,SG_SEXO_ALUNO,SG_UF_AULA,SG_UF_EST2GRAU,SG_UF_FERIAS,SG_UF_NASC_ALUNO,ST_ALUNO,TP_DEFIC_INEP,TP_ESCOLA,TP_INGRES_CURSO,TP_OPCAO_COTAS
66372,2021,10129599724,26,0,I,NaN,2021-06-16,2022-04-25,2021,2003-06-16,1.0,P,B,2,MARINGA,MARINGA,MARINGA,MARINGA,BRASIL,A,M,PR,PR,PR,PR,NaN,NNNNNNNN,P,1,N


In [15]:
df_final[(df_final['DT_NASC_ALUNO'] == '2004-04-12') & (df_final['CD_CURSO'] == 26)]

,ANO_OCORRENCIA,CD_ALUNO,CD_CURSO,CD_ORIGEM_ALUNO,CD_TURNO,DE_NATURALIDADE,DH_MATRICULA,DH_SITUACAO,DT_ANO_INGRES,DT_NASC_ALUNO,LT_ESTADO_CIVIL,LT_MODALIDADE,LT_NACIONALIDADE,LT_RACA,NM_CIDADE_AULA,NM_CIDADE_EST2GRAU,NM_CIDADE_FERIAS,NM_CIDADE_NASC,NM_PAIS_NASC,NU_SEMESTRE2_VEST,SG_SEXO_ALUNO,SG_UF_AULA,SG_UF_EST2GRAU,SG_UF_FERIAS,SG_UF_NASC_ALUNO,ST_ALUNO,TP_DEFIC_INEP,TP_ESCOLA,TP_INGRES_CURSO,TP_OPCAO_COTAS
70071,0,13159171903,26,0,I,NaN,2022-05-16,2022-05-16,2022,2004-04-12,1.0,P,B,1,MARINGA,MARINGA,MARINGA,MARINGA,BRASIL,A,F,PR,PR,PR,PR,NaN,NNNNNNNNNNNNN,P,14,N


### ANO_OCORRENCIA

Não mexi e não iremos utilizar, uma vez que não há o campo da Ocorrência

In [16]:
colunas_a_remover.append('ANO_OCORRENCIA')

### CD_ALUNO

ID do aluno. Mantém como está

### CD_CURSO

Mantém como está.

### CD_ORIGEM_ALUNO

In [17]:
df_final['CD_ORIGEM_ALUNO'].value_counts(dropna=False).sort_index()

CD_ORIGEM_ALUNO
0      7322
1     27404
2      6925
3     17061
4      9967
5      2394
6       147
15        1
Name: count, dtype: int64

Os valores 0 são nulos, verificando outras colunas referentes ao local de nascimento do aluno

In [18]:
def verifica_origem_aluno(row):
    if row['NM_CIDADE_NASC'] == 'MARINGA':
        return 1
    elif row['SG_UF_NASC_ALUNO'] == 'SP':
        return 4
    elif row['SG_UF_NASC_ALUNO'] != 'SP' and row['SG_UF_NASC_ALUNO'] != 'PR':
        return 5
    elif row['NM_PAIS_NASC'] != 'BRASIL':
        return 6
    else:
        return 0

In [19]:
df_final['F_CD_ORIGEM_ALUNO'] = df_final.apply(verifica_origem_aluno, axis=1)

In [20]:
df_final[df_final['CD_ORIGEM_ALUNO'] == 0][colunas_loc_nasc_aluno + ['F_CD_ORIGEM_ALUNO']]

,CD_ORIGEM_ALUNO,LT_NACIONALIDADE,NM_PAIS_NASC,SG_UF_NASC_ALUNO,NM_CIDADE_NASC,SG_UF_FERIAS,NM_CIDADE_FERIAS,F_CD_ORIGEM_ALUNO
1,0,B,BRASIL,PR,MANDAGUARI,NaN,NaN,0
51,0,B,BRASIL,PR,MARINGA,PR,MARINGA,1
809,0,B,BRASIL,PR,LONDRINA,PR,MARIALVA,0
1489,0,B,BRASIL,PR,MARINGA,--,NAO INFORMADO,1
1490,0,B,BRASIL,PR,MARINGA,--,NAO INFORMADO,1
...,...,...,...,...,...,...,...,...
71216,0,B,BRASIL,PR,BRAGANTINA,NaN,MARINGA,0
71217,0,B,BRASIL,PR,CURITIBA,NaN,APUCARANA,0
71218,0,B,NaN,PR,JACAREZINHO,NaN,NaN,6
71219,0,B,NaN,PR,LONDRINA,NaN,NaN,6


Comecei a tratar, mas acho melhor voltar depois de tratar 'CD_ORIGEM_ALUNO', 'LT_NACIONALIDADE', 'NM_CIDADE_NASC', 'NM_PAIS_NASC', 'SG_UF_NASC_ALUNO'

In [21]:
colunas_a_remover.append('CD_ORIGEM_ALUNO')

### CD_TURNO

Mantém como está.

### DE_NATURALIDADE

Comentário do Alison:

> Acho que deveria ser a cidade de nascimento. Acho que este também não é um bom campo, pois também está em desuso, a maioria dos alunos estão com esse campo nulo. O que se usa mais é DE_NAT_ESTRANG o qual é a cidade de nascimento quando a pessoa é estrangeira e o próprio usuário digita como quiser, pois o campo NM_CIDADE_NASC é populado por um selectbox conforme cadastro dos correios e não tem cidades fora do Brasil.

Melhor desconsiderar esse campo.

In [22]:
colunas_a_remover.append('DE_NATURALIDADE')

### DH_MATRICULA

Mantém como está. Ano calendário

### DH_SITUACAO

Mantém como está. Ano calendário

Data da última situação do aluno

A partir de 2007 estará mais confiável. Antes pode ter informação errada. Foi feita uma tratativa para pegar o DH_MATRICULA se o DH_SITUACAO estivesse vazio.


Um caso: aluno entrou em 2014 e estava com a situação em branco, ou seja, matriculado. Quando um aluno tranca e volta, o DH_SITUACAO vai estar com a data da volta, e não da matrícula.

Outro caso: aluno com data de matrícula 2009 e data de ingresso 2006, o errado nesse caso seria o DH_MATRICULA. Antigamente era tudo manual.


### DT_ANO_INGRES

Mantém como está. Ano letivo

### DT_NASC_ALUNO

Vou manter apenas o ano (calendário)

In [23]:
df_final['DT_NASC_ALUNO'].value_counts(dropna=False)

DT_NASC_ALUNO
0001-01-01    62
1991-03-01    29
1990-10-10    28
1990-02-08    24
1994-06-22    23
              ..
1965-01-31     1
1969-03-31     1
1972-01-06     1
1979-12-18     1
1969-01-25     1
Name: count, Length: 11681, dtype: int64

In [24]:
df_final['F_DT_NASC_ALUNO'] = df_final['DT_NASC_ALUNO'].apply(lambda x: x.split('-')[0])

In [25]:
df_final[['DT_NASC_ALUNO', 'F_DT_NASC_ALUNO']]

,DT_NASC_ALUNO,F_DT_NASC_ALUNO
0,1974-07-08,1974
1,1971-02-08,1971
2,1971-07-12,1971
3,1974-05-25,1974
4,1966-07-27,1966
...,...,...
71216,1972-10-16,1972
71217,1981-01-16,1981
71218,2001-07-15,2001
71219,1999-12-13,1999


In [26]:
colunas_a_remover.append('DT_NASC_ALUNO')

### LT_ESTADO_CIVIL

0 ou em branco = NÃO INFORMADO

1 = SOLTEIRO(A)

2 = CASADO(A)

3 = SEPARADO(A)

4 = DIVORCIADO(A)

5 = VIÚVO(A)

6 = OUTROS

In [27]:
df_final['LT_ESTADO_CIVIL'].value_counts(dropna=False).sort_index()

LT_ESTADO_CIVIL
0.0     1204
1.0    64671
2.0     3674
3.0     1001
4.0      278
5.0       15
6.0      146
NaN      232
Name: count, dtype: int64

In [28]:
df_final['LT_ESTADO_CIVIL'].fillna(0.0, inplace=True)

### LT_MODALIDADE

Como só tem um valor, desconsiderar coluna.

In [29]:
colunas_a_remover.append('LT_MODALIDADE')

### LT_NACIONALIDADE

Mantém como está? Vamos usar?

In [30]:
df_final['LT_NACIONALIDADE'].value_counts(dropna=False)

LT_NACIONALIDADE
B      70395
NaN      297
R        216
N        179
E        134
Name: count, dtype: int64

B= BRASILEIRA

E= ESTRANGEIRA

N= BRASILEIRO NATURALIZADO

R= BRASILEIRO NASCIDO NO EXTERIOR

Nulo ou em branco = NÃO INFORMADO

### LT_RACA

Mantém como está

In [31]:
df_final['LT_RACA'].value_counts(dropna=False).sort_index()

LT_RACA
0    20818
1     3171
2    37476
3      177
4     1526
5     8053
Name: count, dtype: int64

0 = NÃO DECLARADA

1 = AMARELO(A), DE ORIGEM ORIENTAL

2 = BRANCO(A)

3 = INDÍGENA OU DE ORIGEM INDÍGENA

4 = NEGRO(A)

5 = PARDO(A)/MUDALTO(A)

Nulo ou em branco = NÃO INFORMADO

### NM_CIDADE_AULA

Desconsiderar. Muitas cidades, deveria ser só as cidades das regiões de cada campus.

In [32]:
df_final.groupby('DT_ANO_INGRES')['NM_CIDADE_NASC'].nunique()

DT_ANO_INGRES
2002     90
2003    157
2004    237
2005    351
2006    418
2007    464
2008    460
2009    578
2010    481
2011    470
2012    414
2013    438
2014    431
2015    404
2016    371
2017    371
2018    368
2019    395
2020    438
2021    447
2022    445
Name: NM_CIDADE_NASC, dtype: int64

In [33]:
colunas_a_remover.append('NM_CIDADE_AULA')

### NM_CIDADE_EST2GRAU, NM_CIDADE_FERIAS

Vamos usar?

Verificar se tem influência com a evasão antes de decidir

Separar campus sede dos outros

### NM_CIDADE_NASC

Vamos usar? Muitos nulos, como proceder?

In [34]:
df_final['NM_CIDADE_NASC'].value_counts(dropna=False)

NM_CIDADE_NASC
MARINGA           17790
NaN               13264
CIANORTE           1789
SAO PAULO          1703
UMUARAMA           1455
                  ...  
LIDIANOPOLIS          1
FARROUPILHA           1
PRES. EPITACIO        1
LIMA                  1
BRAGANTINA            1
Name: count, Length: 1502, dtype: int64

In [35]:
df_final[df_final['NM_PAIS_NASC'] != 'BRASIL']['NM_CIDADE_NASC'].value_counts(dropna=False)

NM_CIDADE_NASC
NaN               1274
MARINGA            198
LONDRINA            26
CIANORTE            25
SAO PAULO           24
                  ... 
JACAREI              1
GUATAMBU             1
URANIA               1
JOAQUIM TAVORA       1
JACAREZINHO          1
Name: count, Length: 401, dtype: int64

### NM_PAIS_NASC

In [36]:
df_final['NM_PAIS_NASC'].value_counts(dropna=False)

NM_PAIS_NASC
BRASIL             68872
NaN                 1751
JAPAO                342
ESTADOS UNIDOS        28
PARAGUAI              25
PORTUGAL              20
GUINE-BISSAU          17
CHINA                 15
ESPANHA               12
ARGENTINA             11
BOLIVIA               11
REPUBLICA DO HA       10
ALEMANHA               9
CHILE                  8
AFEGANISTAO            8
ITALIA                 7
PERU                   7
VENEZUELA              7
ANGOLA                 6
FRANCA                 6
CABO VERDE             5
COLOMBIA               4
TAIWAN                 3
CANADA                 2
BR                     2
INGLATERRA             2
HAITI                  2
S.TOME PRINCIPE        2
GUINE EQUATORIA        2
REP. DO HAITI          2
GANA                   1
GUINE BISSAU           1
BRASILEIRO             1
10\/11\/1999           1
REPUBLICA ARABE        1
COSTA DO MARFIM        1
BRAZIL                 1
27032001               1
FOZ DO IGUACU          1
MOCAMBIQUE  

In [37]:
NM_PAIS_NASC_nan = df_final[df_final['NM_PAIS_NASC'].isna()][colunas_loc_nasc_aluno]
NM_PAIS_NASC_nan['LT_NACIONALIDADE'].value_counts(dropna=False)

LT_NACIONALIDADE
B      1731
NaN      13
E         6
N         1
Name: count, dtype: int64

Preenchendo essa coluna com base na coluna LT_NACIONALIDADE

In [38]:
df_final['F_NM_PAIS_NASC'] = df_final.apply(lambda x: 'BRASIL' if x['LT_NACIONALIDADE'] == 'B' else x['NM_PAIS_NASC'], axis=1)

In [39]:
df_final['F_NM_PAIS_NASC'].value_counts(dropna=False)

F_NM_PAIS_NASC
BRASIL             70717
JAPAO                262
ESTADOS UNIDOS        26
PARAGUAI              22
NaN                   20
GUINE-BISSAU          17
CHINA                 15
PORTUGAL              15
BOLIVIA               11
REPUBLICA DO HA       10
ARGENTINA              9
ESPANHA                9
ALEMANHA               9
VENEZUELA              7
CHILE                  7
PERU                   7
FRANCA                 6
ANGOLA                 6
ITALIA                 6
CABO VERDE             5
COLOMBIA               4
TAIWAN                 3
INGLATERRA             2
HAITI                  2
GUINE EQUATORIA        2
REP. DO HAITI          2
CANADA                 2
S.TOME PRINCIPE        2
MOCAMBIQUE             1
GANA                   1
AUSTRIA                1
REINO UNIDO            1
COSTA DO MARFIM        1
REPUBLICA ARABE        1
GUINE BISSAU           1
JORDANIA               1
BENIN                  1
EUA                    1
GUATEMALA              1
REP.DEM.CO

In [40]:
df_final[df_final['F_NM_PAIS_NASC'].isna()][colunas_loc_nasc_aluno + ['F_NM_PAIS_NASC']]

,CD_ORIGEM_ALUNO,LT_NACIONALIDADE,NM_PAIS_NASC,SG_UF_NASC_ALUNO,NM_CIDADE_NASC,SG_UF_FERIAS,NM_CIDADE_FERIAS,F_NM_PAIS_NASC
128,6,E,NaN,NaN,BISSAU,PR,MARINGA,NaN
129,6,E,NaN,NaN,BISSAU,PR,MARINGA,NaN
17112,6,E,NaN,NaN,GUINE-BISSAU,PR,MARINGA,NaN
17115,6,E,NaN,NaN,NaN,PR,MARINGA,NaN
17116,6,E,NaN,NaN,GUINE-BISSAU,PR,MARINGA,NaN
32721,0,N,NaN,SP,SAO PAULO,SP,BAURU,NaN
51119,0,NaN,NaN,SP,NaN,SP,ADAMANTINA,NaN
51849,0,NaN,NaN,PR,NaN,PR,DOURADINA,NaN
52625,0,NaN,NaN,PR,NaN,PR,MARINGA,NaN
54252,0,NaN,NaN,PR,NaN,PR,APUCARANA,NaN


Ainda restam alguns nulos que podemos preencher com a coluna SG_UF_NASC_ALUNO

In [41]:
df_final['F_NM_PAIS_NASC'] = df_final.apply(lambda x: 'BRASIL' if (not pd.isna(x['SG_UF_NASC_ALUNO'])) and pd.isna(x['F_NM_PAIS_NASC'])  else x['F_NM_PAIS_NASC'], axis=1)

In [42]:
df_final[df_final['F_NM_PAIS_NASC'].isna()][colunas_loc_nasc_aluno + ['F_NM_PAIS_NASC']]

,CD_ORIGEM_ALUNO,LT_NACIONALIDADE,NM_PAIS_NASC,SG_UF_NASC_ALUNO,NM_CIDADE_NASC,SG_UF_FERIAS,NM_CIDADE_FERIAS,F_NM_PAIS_NASC
128,6,E,NaN,NaN,BISSAU,PR,MARINGA,NaN
129,6,E,NaN,NaN,BISSAU,PR,MARINGA,NaN
17112,6,E,NaN,NaN,GUINE-BISSAU,PR,MARINGA,NaN
17115,6,E,NaN,NaN,NaN,PR,MARINGA,NaN
17116,6,E,NaN,NaN,GUINE-BISSAU,PR,MARINGA,NaN
62690,1,E,NaN,NaN,NaN,NaN,NaN,NaN


Existem ainda NaN cuja NM_CIDADE_NASC está BISSAU ou GUINE-BISSAU ainda podemos substituir. Bissau é a capital de Guiné-Bissau, então vou substituir todos esses por GUINE-BISSAU.

In [43]:
df_final['F_NM_PAIS_NASC'] = df_final.apply(lambda x: 'GUINE-BISSAU' if (not pd.isna(x['NM_CIDADE_NASC'])) and pd.isna(x['F_NM_PAIS_NASC'])  else x['F_NM_PAIS_NASC'], axis=1)

In [44]:
df_final[df_final['F_NM_PAIS_NASC'].isna()][colunas_loc_nasc_aluno + ['F_NM_PAIS_NASC']]

,CD_ORIGEM_ALUNO,LT_NACIONALIDADE,NM_PAIS_NASC,SG_UF_NASC_ALUNO,NM_CIDADE_NASC,SG_UF_FERIAS,NM_CIDADE_FERIAS,F_NM_PAIS_NASC
17115,6,E,NaN,NaN,NaN,PR,MARINGA,NaN
62690,1,E,NaN,NaN,NaN,NaN,NaN,NaN


Esses dois NaN que sobraram não tem muito o que fazer...

In [45]:
colunas_a_remover.append('NM_PAIS_NASC')
colunas_loc_nasc_aluno.remove('NM_PAIS_NASC')
colunas_loc_nasc_aluno.append('F_NM_PAIS_NASC')

### NU_SEMESTRE2_VEST

Ver em conjunto com TP_INGRES_CURSO. Talvez usaremos apenas esse campo, que já é basicamente o TP_INGRES_CURSO agrupado.

É o processo seletivo que o candidato fez para se tornar aluno, e atualmente usamos esses valores:

I = Vestibular de Inverno

V = Vestibular de Verão

A = PAS

D = Vestibular EAD

R = Vagas Remanescentes para Candidatos do Vestibular

E = Vagas Remanescentes para quem fez ENEM

G = Vagas Remanescentes para Refugiados ou Imigrantes em Situação de Vulnerabilidade

S = SiSU

In [46]:
df_final['NU_SEMESTRE2_VEST'].value_counts(dropna=False)

NU_SEMESTRE2_VEST
V      31836
I      27573
A       5781
NaN     4141
E       1128
S        373
R        360
G         22
2          7
Name: count, dtype: int64

In [47]:
df_final[['NU_SEMESTRE2_VEST', 'TP_INGRES_CURSO', 'ST_ALUNO']].head(20)

,NU_SEMESTRE2_VEST,TP_INGRES_CURSO,ST_ALUNO
0,I,1,2
1,NaN,6,7
2,V,1,7
3,NaN,6,8
4,NaN,11,9
5,I,14,7
6,NaN,6,2
7,NaN,6,2
8,V,1,2
9,I,1,8


TP_INGRES_CURSO encontrados quando NU_SEMESTRE2_VEST é nulo.

In [48]:
df_final[df_final['NU_SEMESTRE2_VEST'].isna()]['TP_INGRES_CURSO'].value_counts(dropna=False)

TP_INGRES_CURSO
6     1243
32    1113
3      996
5      384
8      105
22      97
7       50
4       49
53      32
26      24
10      22
56       9
12       4
1        4
11       3
19       2
24       1
16       1
29       1
27       1
Name: count, dtype: int64

5 - PORTADOR DE DIPLOMA PARA OUTRO CURSO
6 - PORTADOR DE DIPLOMA PARA NOVA HABILITACAO
32 - NOVA HABILITAÇÃO - CONTINUIDADE

7 - CONVENIO CULTURAL

3 - TRANSFERENCIA EXTERNA
4 - TRANSFERENCIA EX-OFFICIO
8 - TRANSFERENCIA INTERNA DE CURSO
10 - TRANSFERENCIA DE CAMPUS PARA MESMO CURSO
11 - TRANSFERENCIA DE CAMPUS PARA OUTRO CURSO
26 - PERMUTA DE CAMPUS
53 - TRANSFERENCIA INTERNA INDIGENA
56 - TRANSFERENCIA EXTERNA INDIGENA

1 - VESTIBULAR
12 - PROCESSO SELETIVO
16 - VESTIBULAR - 2 CHAMADA EXTRAORDINARIA
19 - VESTIBULAR - 5 CHAMADA EXTRAORDINARIA
22 - CONCURSO VESTIBULAR - INDIGENA
24 - VESTIBULAR - 6 CHAMADA EXTRAORDINARIA
29 - VESTIBULAR - 3 CHAMADA

27 - MATRICULA SUB-JUDICE

In [49]:
portador_diploma = [5, 6, 32]
convenio_cultural = [7]
transferencia = [3, 4, 8, 10, 11, 26, 53, 56]
vestibular = [1, 12, 16, 19, 22, 24, 29]
subjudice = [27]

Verificando o que tem na coluna NU_SEMESTRE2_VEST para cada valor de TP_INGRES_CURSO que foi econtrado com nulo em NU_SEMESTRE2_VEST

In [50]:
df_final[(df_final['NU_SEMESTRE2_VEST'].isna()) & (df_final['TP_INGRES_CURSO'] == 12)]

,ANO_OCORRENCIA,CD_ALUNO,CD_CURSO,CD_ORIGEM_ALUNO,CD_TURNO,DE_NATURALIDADE,DH_MATRICULA,DH_SITUACAO,DT_ANO_INGRES,DT_NASC_ALUNO,LT_ESTADO_CIVIL,LT_MODALIDADE,LT_NACIONALIDADE,LT_RACA,NM_CIDADE_AULA,NM_CIDADE_EST2GRAU,NM_CIDADE_FERIAS,NM_CIDADE_NASC,NM_PAIS_NASC,NU_SEMESTRE2_VEST,SG_SEXO_ALUNO,SG_UF_AULA,SG_UF_EST2GRAU,SG_UF_FERIAS,SG_UF_NASC_ALUNO,ST_ALUNO,TP_DEFIC_INEP,TP_ESCOLA,TP_INGRES_CURSO,TP_OPCAO_COTAS,F_CD_ORIGEM_ALUNO,F_DT_NASC_ALUNO,F_NM_PAIS_NASC
12297,2011,1746001157,12,1,M,BRASILEIRO,2007-02-01,2011-07-19,2007,1983-08-28,1.0,P,B,0,MARINGA,MARINGA,MARINGA,MARINGA,BRASIL,NaN,M,PR,PR,PR,PR,8,SNNNNNNN,GI,12,N,1,1983,BRASIL
15734,2009,1002268555,13,4,M,BRASILEIRA,2007-02-01,2018-09-25,2007,1987-04-18,1.0,P,B,0,RIBEIRO DOS INDIOS,NaN,RIBEIRO DOS INDIOS,LINS,BRASIL,NaN,F,SP,NaN,SP,SP,6,NaN,P,12,N,4,1987,BRASIL
15737,2009,1153058270,13,3,M,BRASILEIRA,2007-02-01,2009-07-21,2007,1972-10-06,2.0,P,B,0,CIANORTE,NaN,CIANORTE,NaN,BRASIL,NaN,F,AC,NaN,PR,PR,8,NaN,GI,12,N,0,1972,BRASIL
15745,2011,1555422179,12,3,M,BRASILEIRA,2007-02-01,2011-12-23,2007,1988-07-25,1.0,P,B,2,CRUZEIRO DO OESTE,UMUARAMA,CRUZEIRO DO OESTE,CRUZEIRO DO OESTE,BRASIL,NaN,F,PR,PR,PR,PR,7,NNNNNNNN,P,12,N,0,1988,BRASIL


In [51]:
df_final[df_final['TP_INGRES_CURSO'].isin(portador_diploma)]['NU_SEMESTRE2_VEST'].value_counts(dropna=False)

NU_SEMESTRE2_VEST
NaN    2740
V       106
Name: count, dtype: int64

In [52]:
df_final[df_final['TP_INGRES_CURSO'].isin(convenio_cultural)]['NU_SEMESTRE2_VEST'].value_counts(dropna=False)

NU_SEMESTRE2_VEST
NaN    50
Name: count, dtype: int64

In [53]:
df_final[df_final['TP_INGRES_CURSO'].isin(transferencia)]['NU_SEMESTRE2_VEST'].value_counts(dropna=False)

NU_SEMESTRE2_VEST
NaN    1240
I       102
V       101
A         3
Name: count, dtype: int64

In [54]:
df_final[df_final['TP_INGRES_CURSO'].isin(vestibular)]['NU_SEMESTRE2_VEST'].value_counts(dropna=False)

NU_SEMESTRE2_VEST
V      25725
I      21177
A       4143
NaN      110
2          3
Name: count, dtype: int64

In [55]:
df_final[df_final['TP_INGRES_CURSO'].isin(subjudice)]['NU_SEMESTRE2_VEST'].value_counts(dropna=False)

NU_SEMESTRE2_VEST
I      2
V      1
NaN    1
Name: count, dtype: int64

In [56]:
dict_balanceamento = {'V': 57, 'I': 46, 'A':10}

In [57]:
def set_nu_semestre2_vest(linha, dict_balanceamento=dict_balanceamento):
    if linha['NU_SEMESTRE2_VEST'] == '2':
        return 'V'
    elif linha['TP_INGRES_CURSO'] in transferencia:
        return 'T'
    elif linha['TP_INGRES_CURSO'] in portador_diploma:
        return 'P'
    elif not pd.isna(linha['NU_SEMESTRE2_VEST']):
        return linha['NU_SEMESTRE2_VEST']
    elif linha['TP_INGRES_CURSO'] in vestibular:
        if dict_balanceamento['V'] > 0:
            dict_balanceamento['V'] -= 1
            return 'V'
        elif dict_balanceamento['I'] > 0:
            dict_balanceamento['I'] -= 1
            return 'I'
        elif dict_balanceamento['A'] > 0:
            dict_balanceamento['A'] -= 1
            return 'A'
    elif linha['TP_INGRES_CURSO'] in convenio_cultural:
        return 'C'
    elif linha['TP_INGRES_CURSO'] in subjudice:
        return 'V'


In [58]:
df_final['F_NU_SEMESTRE2_VEST'] = df_final.apply(set_nu_semestre2_vest, axis=1)

In [59]:
df_final['F_NU_SEMESTRE2_VEST'].value_counts(dropna=False)

F_NU_SEMESTRE2_VEST
V    31694
I    27517
A     5785
P     2846
T     1446
E     1128
S      373
R      360
C       50
G       22
Name: count, dtype: int64

In [60]:
colunas_a_remover.append('NU_SEMESTRE2_VEST')

Criadas 3 categorias novas: 'C' para convenio cultural, 'T' para transferência e 'P' para portador de diploma.
Para as outras, foram atribuídos proporcionalmente aos valores da coluna NU_SEMESTRE2_VEST

### SG_SEXO_ALUNO

Mantém como está

### SG_UF_AULA, SG_UF_EST2GRAU

Vamos usar?

### SG_UF_FERIAS

In [61]:
df_final['SG_UF_FERIAS'].value_counts(dropna=False)

SG_UF_FERIAS
PR     56591
SP      9339
NaN     3443
MS       669
MT       295
MG       238
SC       170
GO        89
RO        88
AC        63
RS        46
RJ        42
BA        40
DF        21
PA        13
TO        12
ES        11
CE         7
MA         6
--         6
P          5
RN         5
PI         4
PB         4
PE         3
AM         3
RR         2
S          2
SE         1
R          1
AL         1
AP         1
Name: count, dtype: int64

In [62]:
df_final['SG_UF_FERIAS'].unique(), df_final['SG_UF_FERIAS'].nunique(dropna=False)

(array(['PR', nan, 'SP', 'MG', 'SC', 'MT', 'GO', 'AC', 'MS', 'RS', 'RO',
        '--', 'S', 'PA', 'P', 'BA', 'PB', 'RJ', 'ES', 'TO', 'DF', 'RN',
        'CE', 'SE', 'MA', 'PE', 'RR', 'R', 'AL', 'PI', 'AM', 'AP'],
       dtype=object),
 32)

In [63]:
estados_a_remover_uf_ferias = ['--', 'S', 'P', 'R']

df_final[df_final['SG_UF_FERIAS'].isin(estados_a_remover_uf_ferias)]['SG_UF_FERIAS'].value_counts(dropna=False)

SG_UF_FERIAS
--    6
P     5
S     2
R     1
Name: count, dtype: int64

In [64]:
df_final[df_final['SG_UF_FERIAS'].isin(estados_a_remover_uf_ferias)]['NM_CIDADE_FERIAS'].value_counts(dropna=False)

NM_CIDADE_FERIAS
NAO INFORMADO        6
S\U001AO PAULO       1
CAMPO MOUR\U001AO    1
JACAREZINHO          1
GUARACI              1
MARINGA              1
AQUIDABAN            1
HOLAMBRA             1
PALMAS               1
Name: count, dtype: int64

In [65]:
df_final[df_final['SG_UF_FERIAS'].isin(estados_a_remover_uf_ferias)][colunas_loc_nasc_aluno + ['NM_CIDADE_FERIAS', 'SG_UF_FERIAS']]

,CD_ORIGEM_ALUNO,LT_NACIONALIDADE,SG_UF_NASC_ALUNO,NM_CIDADE_NASC,SG_UF_FERIAS,NM_CIDADE_FERIAS,F_NM_PAIS_NASC,NM_CIDADE_FERIAS,SG_UF_FERIAS
1489,0,B,PR,MARINGA,--,NAO INFORMADO,BRASIL,NAO INFORMADO,--
1490,0,B,PR,MARINGA,--,NAO INFORMADO,BRASIL,NAO INFORMADO,--
1491,0,B,PR,MARINGA,--,NAO INFORMADO,BRASIL,NAO INFORMADO,--
1857,4,B,SP,NaN,S,S\U001AO PAULO,BRASIL,S\U001AO PAULO,S
2003,3,B,PR,NaN,P,CAMPO MOUR\U001AO,BRASIL,CAMPO MOUR\U001AO,P
2189,3,B,PR,NaN,P,JACAREZINHO,BRASIL,JACAREZINHO,P
8566,3,B,PR,NaN,P,GUARACI,BRASIL,GUARACI,P
9086,1,B,PR,NaN,P,MARINGA,BRASIL,MARINGA,P
11382,1,B,PR,NaN,P,AQUIDABAN,BRASIL,AQUIDABAN,P
16689,4,B,SP,NaN,S,HOLAMBRA,BRASIL,HOLAMBRA,S


In [66]:
def set_SG_UF_FERIAS_ALUNO_1(linha):
    if linha['NM_CIDADE_NASC'] in ['MARINGA', 'CAMPO MOURAO', 'JACAREZINHO', 'PALMAS', 'AQUIDABAN', 'GUARACI']:
        return 'PR'
    elif linha['NM_CIDADE_NASC'] in ['SAO PAULO', 'HOLAMBRA', 'SAO PAULO']:
        return 'SP'
    elif not pd.isna(linha['SG_UF_FERIAS']):
        return linha['SG_UF_FERIAS']
    else:
        return np.nan

In [67]:
df_final['F_SG_UF_FERIAS'] = df_final.apply(set_SG_UF_FERIAS_ALUNO_1, axis=1)

In [68]:
df_final['F_SG_UF_FERIAS'] = df_final['F_SG_UF_FERIAS'].replace({'P':'PR', 'R':'PR', 'S':'SP'})

In [69]:
df_final[df_final['F_SG_UF_FERIAS'].isna()].shape[0]

3272

Preenchidos os estados com uma letra só, colocando a sigla completa

In [70]:
df_final[df_final['NM_CIDADE_FERIAS'] == 'NAO INFORMADO'][colunas_loc_nasc_aluno + ['NM_CIDADE_FERIAS', 'SG_UF_FERIAS', 'F_SG_UF_FERIAS']]

,CD_ORIGEM_ALUNO,LT_NACIONALIDADE,SG_UF_NASC_ALUNO,NM_CIDADE_NASC,SG_UF_FERIAS,NM_CIDADE_FERIAS,F_NM_PAIS_NASC,NM_CIDADE_FERIAS,SG_UF_FERIAS,F_SG_UF_FERIAS
1489,0,B,PR,MARINGA,--,NAO INFORMADO,BRASIL,NAO INFORMADO,--,PR
1490,0,B,PR,MARINGA,--,NAO INFORMADO,BRASIL,NAO INFORMADO,--,PR
1491,0,B,PR,MARINGA,--,NAO INFORMADO,BRASIL,NAO INFORMADO,--,PR
18665,0,B,SP,BAURU,--,NAO INFORMADO,BRASIL,NAO INFORMADO,--,--
22683,0,B,PR,GOIOERE,--,NAO INFORMADO,BRASIL,NAO INFORMADO,--,--
69828,0,B,CE,FORTALEZA,--,NAO INFORMADO,BRASIL,NAO INFORMADO,--,--


In [71]:
sg_ferias_nulo_cidade = df_final[(pd.isna(df_final['F_SG_UF_FERIAS'])) & (~pd.isna(df_final['NM_CIDADE_FERIAS']))][colunas_loc_nasc_aluno + [ 'SG_UF_FERIAS', 'F_SG_UF_FERIAS']]
sg_ferias_nulo_cidade

,CD_ORIGEM_ALUNO,LT_NACIONALIDADE,SG_UF_NASC_ALUNO,NM_CIDADE_NASC,SG_UF_FERIAS,NM_CIDADE_FERIAS,F_NM_PAIS_NASC,SG_UF_FERIAS,F_SG_UF_FERIAS
106,1,B,SP,GUARULHOS,NaN,MARINGA,BRASIL,NaN,NaN
332,4,B,SP,NaN,NaN,VOTUPORANGA,BRASIL,NaN,NaN
706,1,B,PR,NaN,NaN,MARINGA,BRASIL,NaN,NaN
934,2,B,PR,NaN,NaN,SANTA FE,BRASIL,NaN,NaN
1182,3,B,PR,NaN,NaN,PARANACITY,BRASIL,NaN,NaN
...,...,...,...,...,...,...,...,...,...
71203,0,B,RN,MACAU,NaN,MARINGA,BRASIL,NaN,NaN
71205,0,B,PR,LONDRINA,NaN,LONDRINA,BRASIL,NaN,NaN
71208,0,B,RO,PORTO VELHO,NaN,MARINGA,BRASIL,NaN,NaN
71216,0,B,PR,BRAGANTINA,NaN,MARINGA,BRASIL,NaN,NaN


In [72]:
sg_ferias_nulo_cidade['NM_CIDADE_FERIAS'].value_counts(dropna=False)

NM_CIDADE_FERIAS
MARINGA              118
UMUARAMA              20
SARANDI                9
MANDAGUARI             8
APUCARANA              7
                    ... 
FARTURA                1
QUARTO CENTENARIO      1
HERCULANDIA            1
SERTAOZINHO            1
BOM SUCESSO            1
Name: count, Length: 192, dtype: int64

In [73]:
def set_SG_UF_FERIAS_2(linha, df_estados_municipios=df_estados_municipios):
    if pd.isna(linha['F_SG_UF_FERIAS']):
        try:
            return df_estados_municipios[df_estados_municipios['nome'] == linha['NM_CIDADE_FERIAS']]['uf'].values[0]

        except:
            return linha['F_SG_UF_FERIAS']
    else:
        return linha['F_SG_UF_FERIAS']

In [74]:
df_final['F_SG_UF_FERIAS'] = df_final.apply(set_SG_UF_FERIAS_2, axis=1)

In [75]:
df_final[df_final['F_SG_UF_FERIAS'].isna()].shape[0]

2909

Após usar datasets de estados e cidades brasileiras, ainda ficamos com 2909 nulos.

In [76]:
colunas_a_remover.append('SG_UF_FERIAS')
colunas_loc_nasc_aluno.remove('SG_UF_FERIAS')
colunas_loc_nasc_aluno.append('F_SG_UF_FERIAS')

### SG_UF_NASC_ALUNO

In [77]:
df_final['SG_UF_NASC_ALUNO'].value_counts(dropna=False)

SG_UF_NASC_ALUNO
PR     51503
SP     13942
MS      1071
NaN      962
MT       656
MG       634
SC       453
RO       365
RS       332
RJ       324
GO       187
BA       138
PA       126
DF        88
PE        74
CE        65
AM        49
PB        40
ES        37
MA        31
AL        25
RN        25
TO        23
AP        17
PI        15
AC        15
SE        14
RR         6
WP         1
1          1
PY         1
5          1
Name: count, dtype: int64

In [78]:
df_final['SG_UF_NASC_ALUNO'].unique(), len(df_final['SG_UF_NASC_ALUNO'].unique())

(array(['PR', 'SP', 'RS', 'PB', 'MS', 'RJ', 'SC', 'MG', nan, 'MT', 'GO',
        'RO', 'DF', 'PA', 'PE', 'MA', 'BA', 'AL', 'AM', 'PI', 'ES', 'AC',
        '5', 'CE', 'SE', 'AP', 'RN', '1', 'WP', 'TO', 'PY', 'RR'],
       dtype=object),
 32)

Temos alguns valores que não são siglas de estados, logo devem ser tratados

In [79]:
estados_a_remover = ['5', '1', 'WP', 'PY', np.nan]

df_estados_a_remover = df_final[df_final['SG_UF_NASC_ALUNO'].isin(estados_a_remover)][colunas_loc_nasc_aluno]

df_estados_a_remover[df_estados_a_remover['SG_UF_NASC_ALUNO'] == '5']

,CD_ORIGEM_ALUNO,LT_NACIONALIDADE,SG_UF_NASC_ALUNO,NM_CIDADE_NASC,NM_CIDADE_FERIAS,F_NM_PAIS_NASC,F_SG_UF_FERIAS
4074,5,B,5,NaN,COLIDER,BRASIL,MT


In [80]:
df_estados_a_remover[df_estados_a_remover['F_NM_PAIS_NASC'] == 'BRASIL']

,CD_ORIGEM_ALUNO,LT_NACIONALIDADE,SG_UF_NASC_ALUNO,NM_CIDADE_NASC,NM_CIDADE_FERIAS,F_NM_PAIS_NASC,F_SG_UF_FERIAS
133,6,B,NaN,NaN,MARINGA,BRASIL,PR
1014,5,B,NaN,NaN,MARINGA,BRASIL,PR
2596,3,B,NaN,NaN,PRADO FERREIRA,BRASIL,PR
3242,3,B,NaN,NaN,NaN,BRASIL,NaN
3328,0,B,NaN,NaN,NaN,BRASIL,NaN
...,...,...,...,...,...,...,...
71081,0,B,NaN,NaN,MARINGA,BRASIL,PR
71082,0,B,NaN,NaN,MARINGA,BRASIL,PR
71092,0,B,NaN,NaN,MARINGA,BRASIL,PR
71094,0,B,NaN,NaN,NOVA ESPERANCA,BRASIL,PR


In [81]:
df_estados_a_remover[df_estados_a_remover['F_NM_PAIS_NASC'] == 'BRASIL']['SG_UF_NASC_ALUNO'].value_counts(dropna=False)

SG_UF_NASC_ALUNO
NaN    528
5        1
1        1
WP       1
PY       1
Name: count, dtype: int64

531 alunos nascidos no Brasil tem o campo SG_UF_NASC_ALUNO nulo ou com valores inválidos. Todos os valores inválidos não nulos estão com os brasileiros.

In [82]:
df_estados_a_remover[df_estados_a_remover['F_NM_PAIS_NASC'] == 'BRASIL']['NM_CIDADE_NASC'].value_counts(dropna=False)

NM_CIDADE_NASC
NaN                    503
MARINGA                  7
ASSIS CHATEAUBRIAND      2
SAO PAULO                2
ALMADA                   2
IBARAKI                  1
ISESAKI                  1
YUKI-SHI                 1
SUZUKA                   1
SANTIAGO                 1
FUJIYOSHIDA              1
HARUE                    1
ALTO PARANA              1
CIANORTE                 1
FOZ DO IGUACU            1
BILAC                    1
GUAIRA                   1
TOLEDO                   1
CUIABA                   1
NANCY - FRANCA           1
OBU-SHI                  1
Name: count, dtype: int64

In [83]:
def set_SG_UF_NASC_ALUNO_1(linha):
    if linha['NM_CIDADE_NASC'] in ['MARINGA', 'TOLEDO', 'ASSIS CHATEAUBRIAND', 'ALTO PARANA', 'CIANORTE', 'FOZ DO IGUACU', 'GUAIRA', ]:
        return 'PR'
    elif linha['NM_CIDADE_NASC'] in ['SAO PAULO', 'BILAC']:
        return 'SP'
    elif linha['NM_CIDADE_NASC'] in ['CUIABA']:
        return 'MT'
    elif not pd.isna(linha['SG_UF_NASC_ALUNO']):
        return linha['SG_UF_NASC_ALUNO']
    else:
        return np.nan

In [84]:
df_final['F_SG_UF_NASC_ALUNO'] = df_final.apply(set_SG_UF_NASC_ALUNO_1, axis=1)

In [85]:
df_final['F_SG_UF_NASC_ALUNO'].value_counts(dropna=False)

F_SG_UF_NASC_ALUNO
PR     51547
SP     13934
MS      1069
NaN      935
MT       658
MG       632
SC       453
RO       362
RS       331
RJ       324
GO       187
BA       138
PA       126
DF        87
PE        74
CE        65
AM        47
PB        40
ES        37
MA        31
AL        25
RN        25
TO        23
AP        17
PI        15
AC        15
SE        14
RR         6
WP         1
1          1
PY         1
5          1
Name: count, dtype: int64

In [86]:
def set_SG_UF_NASC_ALUNO_2(linha, df_estados_municipios=df_estados_municipios, estados_a_remover=estados_a_remover):
    if pd.isna(linha['F_SG_UF_NASC_ALUNO']):
        try:
            return df_estados_municipios[df_estados_municipios['nome'] == linha['NM_CIDADE_NASC']]['uf'].values[0]

        except:
            return linha['F_SG_UF_NASC_ALUNO']
    else:
        return linha['F_SG_UF_NASC_ALUNO']

In [87]:
df_final['F_SG_UF_NASC_ALUNO'] = df_final.apply(set_SG_UF_NASC_ALUNO_2, axis=1)

In [88]:
df_final[df_final['F_SG_UF_NASC_ALUNO'].isna()].shape[0]

921

In [89]:
colunas_a_remover.append('SG_UF_NASC_ALUNO')
colunas_loc_nasc_aluno.remove('SG_UF_NASC_ALUNO')
colunas_loc_nasc_aluno.append('F_SG_UF_NASC_ALUNO')

Essas colunas de localização ainda posso refinar mais...

### ST_ALUNO

NaN - **NÃO EVASÃO / MATRICULADO** - Matriculado(a)

0 - **EVASÃO / CANCELAMENTO** - Matrícula Cancelada pela UEM

1 - **NÃO EVASÃO / MATRICULADO** Trancado(a)

2 - **EVASÃO / CANCELAMENTO** - Matrícula Cancelada (Pelo aluno)

4 - **EVASÃO / TRANSFERÊNCIA** - Transferido de Curso (Outro campus)

6 - **EVASÃO / TRANSFERÊNCIA** - Transferido de IES

7 - **NÃO EVASÃO / FORMADO** - Formado(a)

8 - **EVASÃO / CANCELAMENTO** - Cancelado por Abandono

9 - **NÃO EVASÃO / JUBILADO** - Jubilado(a)

B - **NÃO EVASÃO / MATRICULADO** - Transferido de Campus

F - **NÃO EVASÃO / MATRICULADO** - Permuta Campus (Mesmo curso)

H - **EVASÃO / TRANSFERÊNCIA** - Permuta Campus (Outro curso)

I - **NÃO EVASÃO / FALECIDO** - Falecido(a)

J - **EVASÃO / CANCELAMENTO** - Matrícula Cancelada (Reprovação por falta por dois anos consecutivos)

K - **EVASÃO / CANCELAMENTO** - Matrícula Cancelada (Reprovação por nota duas vezes consecutivas – Regimento antigo)

M - **NÃO EVASÃO / MATRICULADO** - Em Mobilidade Estudantil

O - **EVASÃO / TRANSFERÊNCIA** - ND - Situação lançada em meados de 2001 para alunos com ocorrência de “Transferência interna de curso”. Pode incluir outras situações

P - **EVASÃO / TRANSFERÊNCIA** - Transferido de Curso (Mesmo campus)



In [90]:
st_evasao = ['0', '2', '4', '6', '8', 'P', 'O', 'K', 'J', 'H']
st_nao_evasao = [np.nan, '1', '9', 'B', 'F', 'M', '7']

st_cancelamento = ['0', '2', '8', 'J', 'K']
st_transferencia = ['4', '6', 'H', 'O', 'P']
st_matriculado = [np.nan, '1', 'B', 'F' , 'M']
st_formado = ['7']
st_jubilado = ['9']
st_falecido = ['I']

In [91]:
df_final[df_final['ST_ALUNO'].isin(st_evasao) == True].shape[0]

26932

In [92]:
def set_BOOL_TRANCADO(linha):
    if linha['ST_ALUNO'] == '1':
        return 1
    return 0

In [93]:
df_final['BOOL_TRANCADO'] = df_final.apply(set_BOOL_TRANCADO, axis=1)

In [94]:
def set_BOOL_ST_ALUNO(linha:pd.Series) -> int:
    if linha['ST_ALUNO'] in st_evasao:
        return 1
    elif linha['ST_ALUNO'] in st_nao_evasao:
        return 0

In [95]:
df_final['BOOL_ST_ALUNO'] = df_final.apply(set_BOOL_ST_ALUNO, axis=1)

In [96]:
def set_MULTI_ST_ALUNO(linha:pd.Series) -> str:
    if linha['ST_ALUNO'] in st_formado:
        return 'FORMADO'
    elif linha['ST_ALUNO'] in st_cancelamento:
        return 'CANCELAMENTO'
    elif linha['ST_ALUNO'] in st_matriculado:
        return 'MATRICULADO'
    elif linha['ST_ALUNO'] in st_jubilado:
        return 'JUBILADO'
    elif linha['ST_ALUNO'] in st_transferencia:
        return 'TRANSFERENCIA'
    elif linha['ST_ALUNO'] in st_falecido:
        return 'FALECIDO'


In [97]:
df_final['MULTI_ST_ALUNO'] = df_final.apply(set_MULTI_ST_ALUNO, axis=1)

In [98]:
df_final['BOOL_ST_ALUNO'].value_counts(dropna=False)

BOOL_ST_ALUNO
0.0    44241
1.0    26932
NaN       48
Name: count, dtype: int64

Esses NaN são os falecidos.

In [99]:
df_final['BOOL_ST_ALUNO'].value_counts(normalize=True) * 100

BOOL_ST_ALUNO
0.0    62.159808
1.0    37.840192
Name: proportion, dtype: float64

In [100]:
pct_evadidos = df_final[df_final['BOOL_ST_ALUNO'] == 1].shape[0] / df_final.shape[0] * 100
pct_nao_evadidos = df_final[df_final['BOOL_ST_ALUNO'] == 0].shape[0] / df_final.shape[0] * 100

print(f'Percentual de evadidos: {pct_evadidos:.2f}%')
print(f'Percentual de não evadidos: {pct_nao_evadidos:.2f}%')

Percentual de evadidos: 37.81%
Percentual de não evadidos: 62.12%


In [101]:
df_final['MULTI_ST_ALUNO'].value_counts(dropna=False)

MULTI_ST_ALUNO
FORMADO          34016
CANCELAMENTO     26086
MATRICULADO       8981
JUBILADO          1244
TRANSFERENCIA      846
FALECIDO            48
Name: count, dtype: int64

In [102]:
df_final['MULTI_ST_ALUNO'].value_counts(normalize=True) * 100

MULTI_ST_ALUNO
FORMADO          47.761194
CANCELAMENTO     36.626838
MATRICULADO      12.610045
JUBILADO          1.746676
TRANSFERENCIA     1.187852
FALECIDO          0.067396
Name: proportion, dtype: float64

In [103]:
pct_formados = df_final[df_final['MULTI_ST_ALUNO'] == 'FORMADO'].shape[0] / df_final.shape[0] * 100
pct_cancelados = df_final[df_final['MULTI_ST_ALUNO'] == 'CANCELAMENTO'].shape[0] / df_final.shape[0] * 100
pct_matriculados = df_final[df_final['MULTI_ST_ALUNO'] == 'MATRICULADO'].shape[0] / df_final.shape[0] * 100
pct_jubilados = df_final[df_final['MULTI_ST_ALUNO'] == 'JUBILADO'].shape[0] / df_final.shape[0] * 100
pct_transferidos = df_final[df_final['MULTI_ST_ALUNO'] == 'TRANSFERENCIA'].shape[0] / df_final.shape[0] * 100
pct_falecidos = df_final[df_final['MULTI_ST_ALUNO'] == 'FALECIDO'].shape[0] / df_final.shape[0] * 100

print(f'Percentual de FORMADOS: {pct_formados:.2f}%')
print(f'Percentual de CANCELADOS: {pct_cancelados:.2f}%')
print(f'Percentual de MATRICULADOS: {pct_matriculados:.2f}%')
print(f'Percentual de JUBILADOS: {pct_jubilados:.2f}%')
print(f'Percentual de TRANSFERIDOS: {pct_transferidos:.2f}%')
print(f'Percentual de FALECIDOS: {pct_falecidos:.2f}%')

Percentual de FORMADOS: 47.76%
Percentual de CANCELADOS: 36.63%
Percentual de MATRICULADOS: 12.61%
Percentual de JUBILADOS: 1.75%
Percentual de TRANSFERIDOS: 1.19%
Percentual de FALECIDOS: 0.07%


Série temporal - evadidos no decorrer do tempo

### TP_DEFIC_INEP

In [104]:
df_final['TP_DEFIC_INEP'].unique()

array(['NNNNNNNN', nan, 'NNNSNNNN', 'NNNNNNNNNNNNN', 'SNNNNNNN',
       'SNSSNNNN', 'NNSNNNNN', 'SNSNNNNN', 'SNNNNNNNNNNNN', 'NNSSNNNN',
       'NNNNNSNN', 'NNNNNNNNSSNNN', 'SNNSNNNN', 'NNNNNNNNSNNNN',
       'SSNNNNNN', 'NSNNNNNN', 'NNSNNNSN', 'SNNNSNNN', 'NNNSNNNNNNNNN',
       'NNNNNNSN', 'NNSNNNNNNNNNN', 'NNNNSNNN', 'NNSNNNSNNNNNN',
       'SSNNNNNNNNNNN', 'NNNNNNNNSNNSN', 'NNNNSNNNNNNNN', 'NNNNNNNNNNNSN',
       'NNNNNNNNNSNNN', 'SNNNNNNNNNNSN', 'NNNNNNNNSSNSN', 'NSNNNNNNNNNNN',
       'SNNNSSNN'], dtype=object)

In [105]:
df_final['TP_DEFIC_INEP'].value_counts(dropna=False)

TP_DEFIC_INEP
NNNNNNNN         49921
NaN              16099
NNNNNNNNNNNNN     4027
SNNNNNNN           873
NNNSNNNN            77
NNSNNNNN            63
SNNNNNNNNNNNN       55
NNNNNNNNNNNSN       13
NNNSNNNNNNNNN       13
NNNNSNNN            12
SNSNNNNN             9
SNNSNNNN             7
NNNNNNNNNSNNN        7
NNSNNNNNNNNNN        6
NNNNNNNNSNNNN        6
NSNNNNNN             6
NNNNSNNNNNNNN        4
NNNNNNNNSSNNN        3
SSNNNNNN             3
NNSNNNSNNNNNN        2
NNSNNNSN             2
SNNNSNNN             2
NNSSNNNN             2
NNNNNNSN             1
SSNNNNNNNNNNN        1
NNNNNNNNSNNSN        1
NNNNNSNN             1
SNSSNNNN             1
SNNNNNNNNNNSN        1
NNNNNNNNSSNSN        1
NSNNNNNNNNNNN        1
SNNNSSNN             1
Name: count, dtype: int64

Como a maioria não tem deficiência, criar coluna boolrana para tem/não tem

In [106]:
df_final['TP_DEFIC_INEP'].fillna('NNNNNNNN').replace({r'[.*S+.*]' : True,
                                                      'NNNNNNNN' : False,
                                                      'NNNNNNNNNNNNN' : False}, regex=True).value_counts(dropna=False)

TP_DEFIC_INEP
False    70047
True      1174
Name: count, dtype: int64

Validado acima

In [107]:
df_final['BOOL_DEFIC_INEP'] = df_final['TP_DEFIC_INEP'].fillna('NNNNNNNN').replace({r'[.*S+.*]' : True,
                                                      'NNNNNNNN' : False,
                                                      'NNNNNNNNNNNNN' : False}, regex=True)

In [108]:
df_final['BOOL_DEFIC_INEP'].value_counts(dropna=False)

BOOL_DEFIC_INEP
False    70047
True      1174
Name: count, dtype: int64

### TP_ESCOLA

In [109]:
df_final['TP_ESCOLA'].value_counts(dropna=False)

TP_ESCOLA
GI     33592
P      27342
E       4467
GP      3392
NaN     1154
SG       776
SP       266
F        107
EX        91
M         34
Name: count, dtype: int64

Os tipos F, E e M não são mais utilizados. Vou agregar todos em uma única categoria 'G'. Quanto aos nulos, não há o que fazer.

In [110]:
df_final['F_TP_ESCOLA'] = df_final['TP_ESCOLA'].replace({'E' : 'G', 'M' : 'G', 'F' : 'G'})

In [111]:
df_final['F_TP_ESCOLA'].value_counts(dropna=False)

F_TP_ESCOLA
GI     33592
P      27342
G       4608
GP      3392
NaN     1154
SG       776
SP       266
EX        91
Name: count, dtype: int64

In [112]:
colunas_a_remover.append('TP_ESCOLA')

### TP_INGRES_CURSO

In [113]:
colunas_a_remover.append('TP_INGRES_CURSO')

Provavelmente não será utilizada da forma como está, muitos valores.

Agrupá-los de alguma forma. A coluna NU_SEMESTRE2_VEST já é essa agrupada.

Valores e grupos de TP_INGRESS_CURSO quando NU_SEMESTRE2_VEST.

5 - PORTADOR DE DIPLOMA PARA OUTRO CURSO
6 - PORTADOR DE DIPLOMA PARA NOVA HABILITACAO
32 - NOVA HABILITAÇÃO - CONTINUIDADE

7 - CONVENIO CULTURAL

3 - TRANSFERENCIA EXTERNA
4 - TRANSFERENCIA EX-OFFICIO
8 - TRANSFERENCIA INTERNA DE CURSO
10 - TRANSFERENCIA DE CAMPUS PARA MESMO CURSO
11 - TRANSFERENCIA DE CAMPUS PARA OUTRO CURSO
26 - PERMUTA DE CAMPUS
53 - TRANSFERENCIA INTERNA INDIGENA
56 - TRANSFERENCIA EXTERNA INDIGENA

1 - VESTIBULAR
12 - PROCESSO SELETIVO
16 - VESTIBULAR - 2 CHAMADA EXTRAORDINARIA
19 - VESTIBULAR - 5 CHAMADA EXTRAORDINARIA
22 - CONCURSO VESTIBULAR - INDIGENA
24 - VESTIBULAR - 6 CHAMADA EXTRAORDINARIA
29 - VESTIBULAR - 3 CHAMADA

27 - MATRICULA SUB-JUDICE

### TP_OPCAO_COTAS

In [114]:
df_final['TP_OPCAO_COTAS'].value_counts(dropna=False)

TP_OPCAO_COTAS
N      62808
S       5862
NaN     2277
O        127
G        126
F         21
Name: count, dtype: int64

Os nulos aqui são não informado. Vou considerar sem cotas, universal

In [115]:
df_final['F_TP_OPCAO_COTAS'] = df_final['TP_OPCAO_COTAS'].fillna('N')

In [116]:
colunas_a_remover.append('TP_OPCAO_COTAS')

Criando uma coluna booleana de cotas

In [117]:
df_final['BOOL_COTAS'] = df_final['F_TP_OPCAO_COTAS'].apply(lambda x: 0 if x == 'N' else 1)

In [118]:
df_final['BOOL_COTAS'].value_counts(dropna=False)

BOOL_COTAS
0    65085
1     6136
Name: count, dtype: int64

# Colunas

In [119]:
df_final.columns

Index(['ANO_OCORRENCIA', 'CD_ALUNO', 'CD_CURSO', 'CD_ORIGEM_ALUNO', 'CD_TURNO',
       'DE_NATURALIDADE', 'DH_MATRICULA', 'DH_SITUACAO', 'DT_ANO_INGRES',
       'DT_NASC_ALUNO', 'LT_ESTADO_CIVIL', 'LT_MODALIDADE', 'LT_NACIONALIDADE',
       'LT_RACA', 'NM_CIDADE_AULA', 'NM_CIDADE_EST2GRAU', 'NM_CIDADE_FERIAS',
       'NM_CIDADE_NASC', 'NM_PAIS_NASC', 'NU_SEMESTRE2_VEST', 'SG_SEXO_ALUNO',
       'SG_UF_AULA', 'SG_UF_EST2GRAU', 'SG_UF_FERIAS', 'SG_UF_NASC_ALUNO',
       'ST_ALUNO', 'TP_DEFIC_INEP', 'TP_ESCOLA', 'TP_INGRES_CURSO',
       'TP_OPCAO_COTAS', 'F_CD_ORIGEM_ALUNO', 'F_DT_NASC_ALUNO',
       'F_NM_PAIS_NASC', 'F_NU_SEMESTRE2_VEST', 'F_SG_UF_FERIAS',
       'F_SG_UF_NASC_ALUNO', 'BOOL_TRANCADO', 'BOOL_ST_ALUNO',
       'MULTI_ST_ALUNO', 'BOOL_DEFIC_INEP', 'F_TP_ESCOLA', 'F_TP_OPCAO_COTAS',
       'BOOL_COTAS'],
      dtype='object')

Idade no ingresso

Morar ou não na cidade do campus

Ser de outros estado



# MEDIDAS

In [147]:
df_merge = pd.read_csv('arquivos\dadosalunos-limpo.csv')

C:\Users\romul\AppData\Local\Temp\ipykernel_15716\761581769.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_merge = pd.read_csv('arquivos\dadosalunos-limpo.csv')


### Idade na matrícula

In [106]:
from datetime import datetime, timedelta

In [107]:
df_final[df_final['DH_MATRICULA'].isna()]

,ANO_OCORRENCIA,CD_ALUNO,CD_CURSO,CD_ORIGEM_ALUNO,CD_TURNO,DE_NATURALIDADE,DH_MATRICULA,DH_SITUACAO,DT_ANO_INGRES,DT_NASC_ALUNO,...,ANO_NASCIMENTO,MESES_ATE_ULTIMA_SITUACAO,NM_CURSO,TP_HABILITACAO,SG_CENTRO,NM_CAMPUS,MAX_ANOS,BOOL_SEDE,LOCAL_FERIAS,LOCAL_NASC


In [108]:
df_final[df_final['DT_NASC_ALUNO'].isna()]

,ANO_OCORRENCIA,CD_ALUNO,CD_CURSO,CD_ORIGEM_ALUNO,CD_TURNO,DE_NATURALIDADE,DH_MATRICULA,DH_SITUACAO,DT_ANO_INGRES,DT_NASC_ALUNO,...,ANO_NASCIMENTO,MESES_ATE_ULTIMA_SITUACAO,NM_CURSO,TP_HABILITACAO,SG_CENTRO,NM_CAMPUS,MAX_ANOS,BOOL_SEDE,LOCAL_FERIAS,LOCAL_NASC


In [109]:
df_final['DT_NASC_ALUNO']

0        1974-07-08
1        1971-07-12
2        1966-07-27
3        1967-10-01
4        1975-12-06
            ...    
66814    1999-12-01
66815    1995-10-11
66816    1972-10-16
66817    1981-01-16
66818    2001-07-15
Name: DT_NASC_ALUNO, Length: 66819, dtype: object

In [110]:
diff = pd.to_datetime(df_final['DH_MATRICULA'], errors='coerce') - pd.to_datetime(df_final['DT_NASC_ALUNO'], errors='coerce')

def a(linha):
    if pd.isna(linha):
        return np.nan

    return np.floor(linha.days / 365)

df_final['IDADE_MATRICULA'] = diff.apply(a)

In [111]:
df_final['IDADE_MATRICULA'].nunique()

52

In [112]:
df_final['IDADE_MATRICULA'].value_counts(dropna=False)

IDADE_MATRICULA
17.0    19435
18.0    16814
19.0     9156
20.0     5095
21.0     3253
22.0     2418
23.0     1834
24.0     1392
25.0     1023
16.0      933
26.0      856
27.0      734
28.0      569
29.0      463
30.0      364
31.0      300
32.0      250
33.0      222
35.0      184
34.0      183
36.0      168
37.0      154
38.0      119
39.0      114
40.0       91
41.0       89
43.0       71
42.0       71
44.0       59
45.0       56
47.0       56
46.0       46
48.0       37
49.0       37
50.0       32
51.0       29
52.0       20
53.0       18
55.0       14
57.0       13
56.0       11
54.0       11
58.0        6
60.0        3
59.0        3
63.0        3
61.0        3
15.0        2
62.0        2
66.0        1
65.0        1
0.0         1
Name: count, dtype: int64

In [113]:
df_final[(df_final['IDADE_MATRICULA'] < 16) | (df_final['IDADE_MATRICULA'] > 70)][['DT_NASC_ALUNO', 'DH_MATRICULA', 'IDADE_MATRICULA']]

,DT_NASC_ALUNO,DH_MATRICULA,IDADE_MATRICULA
23046,1992-08-17,2008-08-04,15.0
32079,1995-01-21,2011-01-16,15.0
65424,2021-09-30,2022-04-26,0.0


In [114]:
df_final[pd.isna(df_final['IDADE_MATRICULA'])][['DT_NASC_ALUNO', 'DH_MATRICULA', 'IDADE_MATRICULA']]

,DT_NASC_ALUNO,DH_MATRICULA,IDADE_MATRICULA


Definir faixa etária

### Ano de nascimento

In [115]:
df_final['ANO_NASCIMENTO'] = df_final['DT_NASC_ALUNO'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').year)

In [116]:
df_final['ANO_NASCIMENTO'].nunique(dropna=False)

64

In [117]:
df_final['ANO_NASCIMENTO'].value_counts(dropna=False)

ANO_NASCIMENTO
1990    3839
1991    3518
1989    3479
1988    3397
1994    3098
        ... 
1950       2
1944       1
1945       1
1943       1
2021       1
Name: count, Length: 64, dtype: int64

In [118]:
df_final[pd.isna(df_final['ANO_NASCIMENTO'])][['DT_NASC_ALUNO', 'DH_MATRICULA', 'ANO_NASCIMENTO']]

,DT_NASC_ALUNO,DH_MATRICULA,ANO_NASCIMENTO


### Curso é ou não no campus sede

In [119]:
df_cursos['BOOL_SEDE'] = df_cursos.apply(lambda x: True if x['NM_CAMPUS'] == 'MARINGA' else False, axis=1)

### Tempo cursado até última situação (em meses)

In [120]:
diff = pd.to_datetime(df_final['DH_SITUACAO'], errors='coerce') - pd.to_datetime(df_final['DH_MATRICULA'], errors='coerce')
diff

0        228 days
1       1466 days
2       1825 days
3       1567 days
4        248 days
           ...   
66814    361 days
66815    333 days
66816    355 days
66817      0 days
66818      0 days
Length: 66819, dtype: timedelta64[ns]

In [121]:
def to_meses(linha):
    if pd.isna(linha):
        return np.nan

    return linha.days / 30

In [122]:
df_final['MESES_ATE_ULTIMA_SITUACAO'] = diff.apply(to_meses)

In [123]:
df_final['MESES_ATE_ULTIMA_SITUACAO'].value_counts(dropna=False)

MESES_ATE_ULTIMA_SITUACAO
0.000000      10305
0.133333        726
59.533333       641
60.466667       615
47.366667       535
              ...  
80.166667         1
55.733333         1
201.566667        1
117.200000        1
8.800000          1
Name: count, Length: 3363, dtype: int64

### CD_ORIGEM_ALUNO

In [124]:
cidades_origem_maringa = df_final[df_final['CD_ORIGEM_ALUNO'] == 1][['NM_CIDADE_NASC', 'NM_CIDADE_FERIAS', 'NM_CIDADE_AULA', 'NM_CIDADE_EST2GRAU']]
cidades_origem_maringa

,NM_CIDADE_NASC,NM_CIDADE_FERIAS,NM_CIDADE_AULA,NM_CIDADE_EST2GRAU
0,MARINGA,MARINGA,MARINGA,MARINGA
1,MARINGA,MARINGA,MARINGA,MARINGA
4,MARINGA,ITAMBE,MARINGA,NaN
8,MARINGA,MARINGA,MARINGA,MARINGA
9,MARINGA,MARINGA,MARINGA,MARINGA
...,...,...,...,...
66722,MARINGA,MARINGA,MARINGA,SARANDI
66726,UBATUBA,CIANORTE,CIANORTE,UBATUBA
66750,UMUARAMA,UMUARAMA,UMUARAMA,UMUARAMA
66781,UMUARAMA,UMUARAMA,UMUARAMA,ALTO PIQUIRI


A ideia era pegar esse campo (CD_ORIGEM_ALUNO) e verificar quais são as cidades que ele considera na região de Maringá para criar este campo para NM_CIDADE_NASC e NM_CIDADE_FERIAS. Porém, ao filtrarmos esse campo para Maringá, vemos entradas que em nenhum dos campos de cidade consta Maringá...

O que ele considera como ORIGEM? Queria responder isso também

Ideias
* Passa as férias na cidade de seu campus? (Usar NM_CIDADE_FERIAS, já que NM_CIDADE_AULA não é útil)
* Aluno nasceu no PR, SP, Outros Estados ou Exterior (SG_UF_NASC_ALUNO)
* Da para fazer um igual ao de cima para SG_UF_FERIAS

### LOCAL_FERIAS e LOCAL_NASCIMENTO

Merge com o df dos cursos para ver o campus

In [125]:
df_merge = df_final.merge(df_cursos, on='CD_CURSO', how='left')
df_merge.head()

,ANO_OCORRENCIA,CD_ALUNO,CD_CURSO,CD_ORIGEM_ALUNO,CD_TURNO,DE_NATURALIDADE,DH_MATRICULA,DH_SITUACAO,DT_ANO_INGRES,DT_NASC_ALUNO,...,MAX_ANOS_x,BOOL_SEDE_x,LOCAL_FERIAS,LOCAL_NASC,NM_CURSO_y,TP_HABILITACAO_y,SG_CENTRO_y,NM_CAMPUS_y,MAX_ANOS_y,BOOL_SEDE_y
0,2014,145457572,3,1,N,BRASILEIRO,2014-01-10,2014-08-26,2014,1974-07-08,...,8,True,1.0,1.0,Letras,LICENCIATURA,CCH,MARINGA,8,True
1,2018,131671183,34,1,V,BRASILEIRO,2015-01-13,2019-01-18,2015,1971-07-12,...,8,True,1.0,1.0,Filosofia,LICENCIATURA,CCH,MARINGA,8,True
2,2009,1748606897,3,5,N,BRASILEIRA,2004-02-01,2009-01-30,2004,1966-07-27,...,8,True,1.0,4.0,Letras,LICENCIATURA,CCH,MARINGA,8,True
3,2020,1204692590,7,3,N,BRASILEIRA,2017-03-09,2021-06-23,2017,1967-10-01,...,8,True,1.0,2.0,Matemática,AMBOS,CCE,MARINGA,8,True
4,2011,1308695326,4,1,M,BRASILEIRA,2011-02-14,2011-10-20,2011,1975-12-06,...,8,True,2.0,1.0,Geografia,AMBOS,CCH,MARINGA,8,True


In [126]:
df_merge[df_merge['F_SG_UF_FERIAS'].isna()].shape[0], df_merge[df_merge['NM_CIDADE_FERIAS'].isna()].shape[0]

(0, 123)

In [127]:
df_merge[df_merge['F_SG_UF_NASC_ALUNO'].isna()].shape[0], df_merge[df_merge['NM_CIDADE_NASC'].isna()].shape[0]

(0, 9545)

In [128]:
df_merge[df_merge['F_SG_UF_FERIAS'].isna()]['F_NM_PAIS_NASC'].value_counts(dropna=False)

Series([], Name: count, dtype: int64)

In [129]:
df_merge[df_merge['F_SG_UF_NASC_ALUNO'].isna()]['F_NM_PAIS_NASC'].value_counts(dropna=False)

Series([], Name: count, dtype: int64)

1 - passa férias / nasceu na cidade do campus

2 - outras cidades do Paraná

3 - estado de São Paulo

4 - outros estados

Esse 5 não se aplica para férias, 29 casos apenas, o que é irrelevante

5 - exterior

In [130]:
df_merge.columns

Index(['ANO_OCORRENCIA', 'CD_ALUNO', 'CD_CURSO', 'CD_ORIGEM_ALUNO', 'CD_TURNO',
       'DE_NATURALIDADE', 'DH_MATRICULA', 'DH_SITUACAO', 'DT_ANO_INGRES',
       'DT_NASC_ALUNO', 'LT_ESTADO_CIVIL', 'LT_MODALIDADE', 'LT_NACIONALIDADE',
       'LT_RACA', 'NM_CIDADE_AULA', 'NM_CIDADE_EST2GRAU', 'NM_CIDADE_FERIAS',
       'NM_CIDADE_NASC', 'NM_PAIS_NASC', 'NU_SEMESTRE2_VEST', 'SG_SEXO_ALUNO',
       'SG_UF_AULA', 'SG_UF_EST2GRAU', 'SG_UF_FERIAS', 'SG_UF_NASC_ALUNO',
       'ST_ALUNO', 'TP_DEFIC_INEP', 'TP_ESCOLA', 'TP_INGRES_CURSO',
       'TP_OPCAO_COTAS', 'F_CD_ORIGEM_ALUNO', 'F_DT_NASC_ALUNO',
       'F_NM_PAIS_NASC', 'F_NU_SEMESTRE2_VEST', 'F_SG_UF_FERIAS',
       'F_SG_UF_NASC_ALUNO', 'BOOL_TRANCADO', 'BOOL_ST_ALUNO',
       'MULTI_ST_ALUNO', 'BOOL_DEFIC_INEP', 'F_TP_ESCOLA', 'F_TP_OPCAO_COTAS',
       'BOOL_COTAS', 'IDADE_MATRICULA', 'ANO_NASCIMENTO',
       'MESES_ATE_ULTIMA_SITUACAO', 'NM_CURSO_x', 'TP_HABILITACAO_x',
       'SG_CENTRO_x', 'NM_CAMPUS_x', 'MAX_ANOS_x', 'BOOL_SEDE_x',

In [131]:
def det_LOCAL_FERIAS(linha):
    if linha['NM_CIDADE_FERIAS'] == linha['NM_CAMPUS']:
        return 1
    elif linha['F_SG_UF_FERIAS'] == 'PR':
        return 2
    elif linha['F_SG_UF_FERIAS'] == 'SP':
        return 3
    elif not pd.isna(linha['F_SG_UF_FERIAS']):
        return 4
    else:
        return np.nan

In [132]:
def det_LOCAL_NASC(linha):
    if linha['NM_CIDADE_NASC'] == linha['NM_CAMPUS']:
        return 1
    elif linha['F_SG_UF_NASC_ALUNO'] == 'PR':
        return 2
    elif linha['F_SG_UF_NASC_ALUNO'] == 'SP':
        return 3
    elif not pd.isna(linha['F_SG_UF_NASC_ALUNO']):
        return 4
    elif linha['F_NM_PAIS_NASC'] != 'BRASIL':
        return 5
    else:
        return np.nan


In [133]:
df_merge['LOCAL_FERIAS'] = df_merge.apply(det_LOCAL_FERIAS, axis=1)
df_merge['LOCAL_NASC'] = df_merge.apply(det_LOCAL_NASC, axis=1)

KeyError: 'NM_CAMPUS'

In [ ]:
df_merge['LOCAL_FERIAS'].value_counts(dropna=False)

LOCAL_FERIAS
1.0    31863
2.0    24982
3.0     9740
NaN     2909
4.0     1727
Name: count, dtype: int64

In [ ]:
df_merge['LOCAL_NASC'].value_counts(dropna=False)

LOCAL_NASC
2.0    32489
1.0    19064
3.0    13937
4.0     4810
NaN      509
5.0      412
Name: count, dtype: int64

In [ ]:
df_merge['F_NM_PAIS_NASC'].value_counts(dropna=False)

F_NM_PAIS_NASC
BRASIL             70731
JAPAO                262
ESTADOS UNIDOS        26
PARAGUAI              22
GUINE-BISSAU          21
CHINA                 15
PORTUGAL              15
BOLIVIA               11
REPUBLICA DO HA       10
ARGENTINA              9
ESPANHA                9
ALEMANHA               9
VENEZUELA              7
CHILE                  7
PERU                   7
FRANCA                 6
ANGOLA                 6
ITALIA                 6
CABO VERDE             5
COLOMBIA               4
TAIWAN                 3
INGLATERRA             2
HAITI                  2
REP. DO HAITI          2
GUINE EQUATORIA        2
CANADA                 2
S.TOME PRINCIPE        2
NaN                    2
URUGUAI                1
AUSTRIA                1
REINO UNIDO            1
COSTA DO MARFIM        1
REPUBLICA ARABE        1
GUINE BISSAU           1
GANA                   1
MOCAMBIQUE             1
EUA                    1
GUATEMALA              1
REP.DEM.CONGO          1
HONDURAS  

### Gerações Completas

In [148]:
df_merge.shape[0]

66819

In [149]:
df_merge['GERACAO_COMPLETA'] = df_merge.apply(lambda x: 1 if (x['DT_ANO_INGRES'] + x['MAX_ANOS']) <= 2022 else 0, axis=1)

In [150]:
df_merge['GERACAO_COMPLETA'].value_counts(dropna=False)

GERACAO_COMPLETA
1    42249
0    24570
Name: count, dtype: int64

In [151]:
df_merge[df_merge['GERACAO_COMPLETA'] == 1]['MULTI_ST_ALUNO'].value_counts()


MULTI_ST_ALUNO
FORMADO          25987
CANCELAMENTO     14593
JUBILADO           982
TRANSFERENCIA      545
MATRICULADO        142
Name: count, dtype: int64

In [152]:
df_merge[df_merge['GERACAO_COMPLETA'] == 0]['MULTI_ST_ALUNO'].value_counts()


MULTI_ST_ALUNO
CANCELAMENTO     9010
MATRICULADO      8644
FORMADO          6562
TRANSFERENCIA     185
JUBILADO          169
Name: count, dtype: int64

Não deveria ter jubilado aqui... vou filtrar jubilados que não são geração completa e matriculados que são geração completa

In [153]:
df_merge.drop(df_merge[(df_merge['GERACAO_COMPLETA'] == 1) & 
                       (df_merge['MULTI_ST_ALUNO'] == 'MATRICULADO')].index, 
                       inplace=True)

In [154]:
df_merge.drop(df_merge[(df_merge['GERACAO_COMPLETA'] == 0) & 
                       (df_merge['MULTI_ST_ALUNO'] == 'JUBILADO')].index, 
              inplace=True)

### Número da Geração

In [155]:
df_merge.shape[0]

66508

In [156]:
df_merge['NUMERO_GERACAO'] = df_merge[df_merge['GERACAO_COMPLETA'] == 1]['DT_ANO_INGRES'] % 2001

In [157]:
df_merge['NUMERO_GERACAO'].fillna(0.0, inplace=True)

In [158]:
df_merge['NUMERO_GERACAO'].value_counts(dropna=False)

NUMERO_GERACAO
0.0     24401
8.0      4078
7.0      4061
10.0     3558
9.0      3503
12.0     3498
11.0     3358
6.0      3178
5.0      2998
4.0      2841
3.0      2749
2.0      2723
13.0     2639
1.0      2201
14.0      607
15.0       63
16.0       52
Name: count, dtype: int64

# Final

In [159]:
df_merge.columns

Index(['ANO_OCORRENCIA', 'CD_ALUNO', 'CD_CURSO', 'CD_ORIGEM_ALUNO', 'CD_TURNO',
       'DE_NATURALIDADE', 'DH_MATRICULA', 'DH_SITUACAO', 'DT_ANO_INGRES',
       'DT_NASC_ALUNO', 'LT_ESTADO_CIVIL', 'LT_MODALIDADE', 'LT_NACIONALIDADE',
       'LT_RACA', 'NM_CIDADE_AULA', 'NM_CIDADE_EST2GRAU', 'NM_CIDADE_FERIAS',
       'NM_CIDADE_NASC', 'NM_PAIS_NASC', 'NU_SEMESTRE2_VEST', 'SG_SEXO_ALUNO',
       'SG_UF_AULA', 'SG_UF_EST2GRAU', 'SG_UF_FERIAS', 'SG_UF_NASC_ALUNO',
       'ST_ALUNO', 'TP_DEFIC_INEP', 'TP_ESCOLA', 'TP_INGRES_CURSO',
       'TP_OPCAO_COTAS', 'F_CD_ORIGEM_ALUNO', 'F_DT_NASC_ALUNO',
       'F_NM_PAIS_NASC', 'F_NU_SEMESTRE2_VEST', 'F_SG_UF_FERIAS',
       'F_SG_UF_NASC_ALUNO', 'BOOL_TRANCADO', 'BOOL_ST_ALUNO',
       'MULTI_ST_ALUNO', 'BOOL_DEFIC_INEP', 'F_TP_ESCOLA', 'F_TP_OPCAO_COTAS',
       'BOOL_COTAS', 'IDADE_MATRICULA', 'ANO_NASCIMENTO',
       'MESES_ATE_ULTIMA_SITUACAO', 'NM_CURSO', 'TP_HABILITACAO', 'SG_CENTRO',
       'NM_CAMPUS', 'MAX_ANOS', 'BOOL_SEDE', 'LOCAL_FERI

In [160]:
colunas_relevantes = ['CD_ALUNO',
                      'CD_CURSO',
                      'CD_TURNO',
                      'DH_MATRICULA',
                      'DH_SITUACAO',
                      'DT_ANO_INGRES',
                      'DT_NASC_ALUNO',
                      'LT_ESTADO_CIVIL',
                      'LT_RACA',
                      'F_NU_SEMESTRE2_VEST',
                      'TP_INGRES_CURSO',
                      'SG_SEXO_ALUNO',
                      'F_NM_PAIS_NASC',
                      'F_SG_UF_NASC_ALUNO',
                      'F_SG_UF_FERIAS',
                      'BOOL_TRANCADO',
                      'BOOL_ST_ALUNO',
                      'MULTI_ST_ALUNO',
                      'BOOL_DEFIC_INEP',
                      'F_TP_ESCOLA',
                      'F_TP_OPCAO_COTAS',
                      'BOOL_COTAS',
                      'IDADE_MATRICULA',
                      'ANO_NASCIMENTO',
                      'MESES_ATE_ULTIMA_SITUACAO',
                      'LOCAL_FERIAS',
                      'LOCAL_NASC',
                      'NM_CURSO',
                      'TP_HABILITACAO',
                      'SG_CENTRO',
                      'NM_CAMPUS',
                      'MAX_ANOS',
                      'GERACAO_COMPLETA',
                      'NUMERO_GERACAO',
                      'BOOL_SEDE']
len(colunas_relevantes)


35

In [161]:
df_merge.dropna(subset=colunas_relevantes, inplace=True)

In [ ]:
# df_merge[colunas_relevantes].apply(lambda x: x.nunique(dropna=False)).reset_index().to_excel('colunas_unicos.xlsx')

In [162]:
df_merge['MULTI_ST_ALUNO'].value_counts(dropna=False)

MULTI_ST_ALUNO
FORMADO          32549
CANCELAMENTO     23603
MATRICULADO       8644
JUBILADO           982
TRANSFERENCIA      730
Name: count, dtype: int64

In [163]:
df_merge = df_merge[df_merge['MULTI_ST_ALUNO'] != 'FALECIDO']

In [164]:
df_merge[colunas_relevantes]

,CD_ALUNO,CD_CURSO,CD_TURNO,DH_MATRICULA,DH_SITUACAO,DT_ANO_INGRES,DT_NASC_ALUNO,LT_ESTADO_CIVIL,LT_RACA,F_NU_SEMESTRE2_VEST,...,LOCAL_FERIAS,LOCAL_NASC,NM_CURSO,TP_HABILITACAO,SG_CENTRO,NM_CAMPUS,MAX_ANOS,GERACAO_COMPLETA,NUMERO_GERACAO,BOOL_SEDE
0,145457572,3,N,2014-01-10,2014-08-26,2014,1974-07-08,1.0,4,I,...,1.0,1.0,Letras,LICENCIATURA,CCH,MARINGA,8,1,13.0,True
1,131671183,34,V,2015-01-13,2019-01-18,2015,1971-07-12,1.0,5,V,...,1.0,1.0,Filosofia,LICENCIATURA,CCH,MARINGA,8,0,0.0,True
2,1748606897,3,N,2004-02-01,2009-01-30,2004,1966-07-27,2.0,0,T,...,1.0,4.0,Letras,LICENCIATURA,CCH,MARINGA,8,1,3.0,True
3,1204692590,7,N,2017-03-09,2021-06-23,2017,1967-10-01,4.0,2,I,...,1.0,2.0,Matemática,AMBOS,CCE,MARINGA,8,0,0.0,True
4,1308695326,4,M,2011-02-14,2011-10-20,2011,1975-12-06,2.0,2,P,...,2.0,1.0,Geografia,AMBOS,CCH,MARINGA,8,1,10.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66814,13366520600,3,M,2022-07-14,2023-07-10,2022,1999-12-01,1.0,0,P,...,1.0,4.0,Letras,LICENCIATURA,CCH,MARINGA,8,0,0.0,True
66815,11333398558,8,M,2022-07-15,2023-06-13,2022,1995-10-11,0.0,0,A,...,2.0,2.0,Administração,BACHARELADO,CSA,MARINGA,8,0,0.0,True
66816,19881374013,13,N,2022-07-20,2023-07-10,2022,1972-10-16,3.0,0,T,...,1.0,2.0,Pedagogia,LICENCIATURA,CCH,MARINGA,8,0,0.0,True
66817,14796067716,42,I,2022-07-29,2022-07-29,2022,1981-01-16,2.0,0,T,...,2.0,2.0,Música,BACHARELADO,CCH,MARINGA,6,0,0.0,True


In [165]:
df_merge.shape[0]

66508

In [166]:
df_final.shape[0] - df_merge.shape[0]

311

In [167]:
df_final['BOOL_ST_ALUNO'].value_counts(), df_final['BOOL_ST_ALUNO'].value_counts(normalize=True) * 100

(BOOL_ST_ALUNO
 0.0    42486
 1.0    24333
 Name: count, dtype: int64,
 BOOL_ST_ALUNO
 0.0    63.583711
 1.0    36.416289
 Name: proportion, dtype: float64)

In [168]:
df_merge['BOOL_ST_ALUNO'].value_counts(), df_merge['BOOL_ST_ALUNO'].value_counts(normalize=True) * 100

(BOOL_ST_ALUNO
 0.0    42175
 1.0    24333
 Name: count, dtype: int64,
 BOOL_ST_ALUNO
 0.0    63.413424
 1.0    36.586576
 Name: proportion, dtype: float64)

In [169]:
df_final['MULTI_ST_ALUNO'].value_counts(), df_final['MULTI_ST_ALUNO'].value_counts(normalize=True) * 100

(MULTI_ST_ALUNO
 FORMADO          32549
 CANCELAMENTO     23603
 MATRICULADO       8786
 JUBILADO          1151
 TRANSFERENCIA      730
 Name: count, dtype: int64,
 MULTI_ST_ALUNO
 FORMADO          48.712193
 CANCELAMENTO     35.323785
 MATRICULADO      13.148955
 JUBILADO          1.722564
 TRANSFERENCIA     1.092504
 Name: proportion, dtype: float64)

In [170]:
df_merge['MULTI_ST_ALUNO'].value_counts(), df_merge['MULTI_ST_ALUNO'].value_counts(normalize=True) * 100

(MULTI_ST_ALUNO
 FORMADO          32549
 CANCELAMENTO     23603
 MATRICULADO       8644
 JUBILADO           982
 TRANSFERENCIA      730
 Name: count, dtype: int64,
 MULTI_ST_ALUNO
 FORMADO          48.939977
 CANCELAMENTO     35.488964
 MATRICULADO      12.996933
 JUBILADO          1.476514
 TRANSFERENCIA     1.097612
 Name: proportion, dtype: float64)

In [171]:
# df_merge['NM_CURSO'].value_counts().reset_index().to_excel('/alunos-curso.xlsx')

### Dropando estados errado

In [172]:
df_merge['F_SG_UF_NASC_ALUNO'].unique()

array(['PR', 'RS', 'SP', 'PB', 'MS', 'RJ', 'SC', 'MG', 'GO', 'DF', 'PA',
       'MT', 'PE', 'MA', 'RO', 'BA', 'AL', 'AM', 'PI', 'ES', 'AC', 'CE',
       'SE', 'AP', 'RN', 'TO', 'RR'], dtype=object)

In [173]:
df_merge[df_merge['F_SG_UF_NASC_ALUNO'] == '5']

,ANO_OCORRENCIA,CD_ALUNO,CD_CURSO,CD_ORIGEM_ALUNO,CD_TURNO,DE_NATURALIDADE,DH_MATRICULA,DH_SITUACAO,DT_ANO_INGRES,DT_NASC_ALUNO,...,NM_CURSO,TP_HABILITACAO,SG_CENTRO,NM_CAMPUS,MAX_ANOS,BOOL_SEDE,LOCAL_FERIAS,LOCAL_NASC,GERACAO_COMPLETA,NUMERO_GERACAO


In [174]:
df_merge.drop(4074, inplace=True)

In [175]:
df_merge[df_merge['F_SG_UF_NASC_ALUNO'] == 'WP']

,ANO_OCORRENCIA,CD_ALUNO,CD_CURSO,CD_ORIGEM_ALUNO,CD_TURNO,DE_NATURALIDADE,DH_MATRICULA,DH_SITUACAO,DT_ANO_INGRES,DT_NASC_ALUNO,...,NM_CURSO,TP_HABILITACAO,SG_CENTRO,NM_CAMPUS,MAX_ANOS,BOOL_SEDE,LOCAL_FERIAS,LOCAL_NASC,GERACAO_COMPLETA,NUMERO_GERACAO


In [176]:
df_merge.drop(14023, inplace=True)

In [177]:
df_merge['F_SG_UF_FERIAS'].unique()

array(['PR', 'SP', 'MG', 'SC', 'MT', 'GO', 'AC', 'MS', 'RS', 'RO', 'PA',
       'BA', 'PB', 'RJ', 'ES', 'TO', 'DF', 'RN', 'CE', 'SE', 'MA', 'PE',
       'RR', 'AL', 'PI', 'AM', 'AP'], dtype=object)

In [178]:
df_merge[df_merge['F_SG_UF_FERIAS'] == '--']

,ANO_OCORRENCIA,CD_ALUNO,CD_CURSO,CD_ORIGEM_ALUNO,CD_TURNO,DE_NATURALIDADE,DH_MATRICULA,DH_SITUACAO,DT_ANO_INGRES,DT_NASC_ALUNO,...,NM_CURSO,TP_HABILITACAO,SG_CENTRO,NM_CAMPUS,MAX_ANOS,BOOL_SEDE,LOCAL_FERIAS,LOCAL_NASC,GERACAO_COMPLETA,NUMERO_GERACAO


In [180]:
df_merge.drop([18665, 22683], inplace=True)

### Visualizações finais

In [181]:
df_merge.shape[0]

66504

In [182]:
df_merge['DT_ANO_INGRES'].min(), df_merge['DT_ANO_INGRES'].max()

(2002, 2022)

In [183]:
df_merge

,ANO_OCORRENCIA,CD_ALUNO,CD_CURSO,CD_ORIGEM_ALUNO,CD_TURNO,DE_NATURALIDADE,DH_MATRICULA,DH_SITUACAO,DT_ANO_INGRES,DT_NASC_ALUNO,...,NM_CURSO,TP_HABILITACAO,SG_CENTRO,NM_CAMPUS,MAX_ANOS,BOOL_SEDE,LOCAL_FERIAS,LOCAL_NASC,GERACAO_COMPLETA,NUMERO_GERACAO
0,2014,145457572,3,1,N,BRASILEIRO,2014-01-10,2014-08-26,2014,1974-07-08,...,Letras,LICENCIATURA,CCH,MARINGA,8,True,1.0,1.0,1,13.0
1,2018,131671183,34,1,V,BRASILEIRO,2015-01-13,2019-01-18,2015,1971-07-12,...,Filosofia,LICENCIATURA,CCH,MARINGA,8,True,1.0,1.0,0,0.0
2,2009,1748606897,3,5,N,BRASILEIRA,2004-02-01,2009-01-30,2004,1966-07-27,...,Letras,LICENCIATURA,CCH,MARINGA,8,True,1.0,4.0,1,3.0
3,2020,1204692590,7,3,N,BRASILEIRA,2017-03-09,2021-06-23,2017,1967-10-01,...,Matemática,AMBOS,CCE,MARINGA,8,True,1.0,2.0,0,0.0
4,2011,1308695326,4,1,M,BRASILEIRA,2011-02-14,2011-10-20,2011,1975-12-06,...,Geografia,AMBOS,CCH,MARINGA,8,True,2.0,1.0,1,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66814,2023,13366520600,3,0,M,NaN,2022-07-14,2023-07-10,2022,1999-12-01,...,Letras,LICENCIATURA,CCH,MARINGA,8,True,1.0,4.0,0,0.0
66815,2022,11333398558,8,3,M,NaN,2022-07-15,2023-06-13,2022,1995-10-11,...,Administração,BACHARELADO,CSA,MARINGA,8,True,2.0,2.0,0,0.0
66816,2023,19881374013,13,0,N,NaN,2022-07-20,2023-07-10,2022,1972-10-16,...,Pedagogia,LICENCIATURA,CCH,MARINGA,8,True,1.0,2.0,0,0.0
66817,0,14796067716,42,0,I,NaN,2022-07-29,2022-07-29,2022,1981-01-16,...,Música,BACHARELADO,CCH,MARINGA,6,True,2.0,2.0,0,0.0


### Ajustes nos tipos das colunas

In [3]:
df_merge = pd.read_csv('arquivos\dadosalunos-limpo.csv')

C:\Users\romulo.mincache\AppData\Local\Temp\ipykernel_26492\761581769.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_merge = pd.read_csv('arquivos\dadosalunos-limpo.csv')


In [4]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66504 entries, 0 to 66503
Data columns (total 56 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ANO_OCORRENCIA             66504 non-null  int64  
 1   CD_ALUNO                   66504 non-null  int64  
 2   CD_CURSO                   66504 non-null  int64  
 3   CD_ORIGEM_ALUNO            66504 non-null  int64  
 4   CD_TURNO                   66504 non-null  object 
 5   DE_NATURALIDADE            46761 non-null  object 
 6   DH_MATRICULA               66504 non-null  object 
 7   DH_SITUACAO                66504 non-null  object 
 8   DT_ANO_INGRES              66504 non-null  int64  
 9   DT_NASC_ALUNO              66504 non-null  object 
 10  LT_ESTADO_CIVIL            66504 non-null  float64
 11  LT_MODALIDADE              66504 non-null  object 
 12  LT_NACIONALIDADE           66247 non-null  object 
 13  LT_RACA                    66504 non-null  int

In [15]:
df_merge['SG_CENTRO'].replace(['CCH  ', 'CCE  ', 'CSA  ', 'CTC  ', 'CCS  ', 'CCA  ', 'CCB  '], ['CCH', 'CCE', 'CSA', 'CTC', 'CCS', 'CCA', 'CCB'], inplace=True)

### Gerando arquivo

In [184]:
df_merge.to_csv('arquivos/dadosalunos-limpo.csv', index=False)
df_merge.to_excel('arquivos/dadosalunos-limpo.xlsx', index=False)